In [5]:
import matplotlib.pyplot as plt
import numpy as np
from case_studies import *
from scipy.optimize import minimize
from case_studies import *

def calculate_x0(A,b,x):
    return np.array(x - np.linalg.pinv(A)@(A@x + b))

def SteepestDescent(x0, f, df, A, dims, c1, rho, tol, maxiter):
    x = x0
    x_list = []
    x_list.append(x)

    beta = 1
    A_AT = np.dot(A,A.T)
    A_ATinv = np.linalg.inv(A_AT)
    M = np.identity(dims) - A.T @ A_ATinv @ A
    
    for i in range(maxiter):          
        p = -np.dot(M,df(x))
        alpha = backtrack(x, f, df, p, c1, rho, beta)   
        x = x + alpha * p
        beta = alpha/rho
        x_list.append(x)
        # check if the norm of the gradient is smaller than the tolerance
        if np.linalg.norm(df(x)) < tol:
            break
    return x_list

def backtrack(x, f, df, p_k, c1, rho, beta_k):
  alpha = beta_k
  while f(x+alpha*p_k) > f(x) + c1 * alpha * p_k @ df(x):
    alpha = rho * alpha
  return alpha

In [6]:
def generate_A(rank,dim):
    while True:
        A = np.random.randint(low=10, high=100, size=(rank, dim))
        if np.linalg.matrix_rank(A) == rank:
            return np.reshape(A,(rank,dim))
        
def generate_b(rank):
    return np.random.randint(low=10, high=100,size = rank)


def generate_x0(A,b,dim):
    x = np.ones(dim)
    return calculate_x0(A,b,x)

In [10]:
dim = 1000

ranks = [100,200,300,400]
maxiter = 50

for rank in ranks:
    A = generate_A(rank,dim)
    b = generate_b(rank)
    x0 = generate_x0(A,b,dim)
    
    x_list = SteepestDescent(x0, f1, df1, A, dim, 0.01, 0.5, 0.01, maxiter)
    
    cons = [{"type": "ineq", "fun": lambda x: A @ x - b}]
    x_list_scipy = minimize(f1, x0, constraints = cons)

    print(np.linalg.norm(x_list[-1]-x_list_scipy.x))
    


0.7987021559758493
0.8608063860924869


In [ ]:
A = np.array([[1,2],[0,1]])
b = np.array([1,3])
maxiter = 100
dim = 2
x0 = calculate_x0(A, b, np.array([100,-100]))
print('x0:' + str(x0))

x_list = SteepestDescent(x0, f1, df1, A, dim, 0.1, 0.5, 0.01, maxiter)

x0:[ 5. -3.]


In [ ]:
cons = [{"type": "ineq", "fun": lambda x: A @ x - b}]
x_list_scipy = minimize(f1, x0, constraints = cons)

In [ ]:
x_list[-1]

array([ 5., -3.])

In [ ]:
x_list_scipy.x

array([-3.82314266e-07,  3.00000000e+00])